# Get the top subreddits by subscriber count
Data From: 2021-11-19

Data Source: https://frontpagemetrics.com/list-all-subreddits

New Data Stored In: top_subreddits.json

In [11]:
import gzip
from csv import reader
from collections import defaultdict

In [12]:
# Will map a subreddit to its popularity
subreddit_popularity = defaultdict(int)

# The total amount of subreddits that will be returned
n = 500

In [13]:
# Open the subreddit popularity data
with gzip.open("../data/2021-11-19.csv.gz", "rt", errors="ignore") as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    
    # Skip the header
    next(csv_reader)
    
    # Iterate over each row in the csv using reader object
    for row in csv_reader:
        subreddit = row[0]
        subscribers = row[-1]
        
        subreddit_popularity[subreddit] = subscribers

In [14]:
# Sort the subreddits by their subscriber count
subreddit_popularity = sorted(subreddit_popularity.items(), key=lambda x: int(x[1]), reverse=True)

In [15]:
file = open("../data/top_subreddits.json", "w")
file.write("{\n")

for i, subreddit in enumerate(subreddit_popularity[:n]):
    if i < (n - 1):
        file.write("\t\"{}\":{},\n".format(subreddit[0], subreddit[1]))
    else:
        file.write("\t\"{}\":{}\n".format(subreddit[0], subreddit[1]))
        
file.write("}\n")
file.close()